# Test notebook for JWebbinars
## Running the pipeline on NIRISS simulated data

Test environment for the JWebbinars with just
- jwst==0.17.1
- jdaviz

Pinned CRDS version - 0645

In [ ]:
%env CRDS_CONTEXT=jwst_0645.pmap

#system general
import os
import fnmatch
import sys
import urllib.request

#plotting and tabling
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import astropy
from astropy.io import fits
from astropy.io import ascii
import astropy.wcs as pywcs
import numpy as np
import h5py
import pandas as pd

#pipeline
import jwst
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline
#wcs
from jwst import assign_wcs
from jwst.assign_wcs import assign_wcs_step
#datamodels
from jwst.datamodels import image, DrizProductModel, WavelengthrangeModel

In [ ]:
print('Versions')
print('astropy:',astropy.__version__)
print('pipeline:',jwst.__version__)
print('numpy:',np.__version__)

In [ ]:
#define a plotting funciton for the mirage images
def show(array,title,min=0,max=1000):
    plt.figure(figsize=(12,12))
    plt.imshow(array,clim=(min,max))
    plt.title(title)
    plt.colorbar().set_label('DN/s')

## File names

In [ ]:
#starting files
uncalfile = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/jwebbinar_test/basic_F150W_direct_uncal.fits"
uncalspecfileC = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/jwebbinar_test/basic_F150W_WFSSC_uncal.fits"
uncalspecfileR = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/jwebbinar_test/basic_F150W_WFSSR_uncal.fits"

#the json files need to be downloaded
associationClink = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/jwebbinar_test/basic_F150W_WFSSC_stage2.json"
associationC = "basic_F150W_WFSSC_stage2.json"
associationRlink = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/jwebbinar_test/basic_F150W_WFSSR_stage2.json"
associationR = "basic_F150W_WFSSR_stage2.json"
urllib.request.urlretrieve(associationClink, associationC)
urllib.request.urlretrieve(associationRlink, associationR)

#files that will be produced
ratefile = "./basic_F150W_direct_rate.fits"
calfile = "./basic_F150W_direct_cal.fits"
catfile = "./basic_F150W_direct_cat.ecsv"
ratespecfileC = "./basic_F150W_WFSSC_rate.fits"
calspecfileC = "basic_F150W_WFSSC_cal.fits"
x1dspecfileC = "./basic_F150W_WFSSC_x1d.fits"
ratespecfileR = "./basic_F150W_WFSSR_rate.fits"
calspecfileR = "./basic_F150W_WFSSR_cal.fits"
x1dspecfileR = "./basic_F150W_WFSSR_x1d.fits"

## Run pipeline

In [ ]:
#check original header
uncalimaged = fits.open(uncalfile)
print(uncalimaged.info())
uncalimaged[0].header

## Run pipeline stage1, 2, and 3 on uncal direct image ##

**Config files are saved locally**

In [ ]:
stage1 = Detector1Pipeline.call(uncalfile, config_file='./configfiles/calwebb_detector1.cfg', save_results=True)

In [ ]:
#plot rate image
rateimage = fits.open(ratefile)

print(rateimage.info())

plt.figure(figsize=(12,12))
plt.imshow((rateimage[1].data),clim=(-1,2))
plt.xlim(1000,1100)
plt.ylim(1000,1100)
plt.title("Science image")
plt.colorbar()

plt.figure(figsize=(12,12))
plt.imshow((rateimage[3].data),clim=(0,10))
plt.xlim(1000,1100)
plt.ylim(1000,1100)
plt.title("DQ array")
plt.colorbar()

In [ ]:
stage2 = Image2Pipeline.call(ratefile, config_file='./configfiles/calwebb_image2.cfg', save_results=True)

In [ ]:
stage3 = Image3Pipeline.call(calfile, config_file='./configfiles/calwebb_image3.cfg', save_results=True)

## Check catalog ##

In [ ]:
#plot the sources obtained
cat = pd.read_table(catfile,comment='#',sep='\s+')
cat.head()

Plot sources on cal images.

In [ ]:
#plot cal image and detected sources
calimage = fits.open(calfile)
plt.figure(figsize=(12,12))
plt.imshow((calimage[1].data),clim=(0,2))
plt.xlim(0,2048)
plt.ylim(0,2048)
plt.colorbar()

plt.scatter(cat['xcentroid'],cat['ycentroid'],s=80, facecolors='none', edgecolors='r')
for i in range (0,len(cat)):
    plt.annotate(cat.loc[i,'id'], (cat.loc[i,'xcentroid'], cat.loc[i,'ycentroid']),color='white')
plt.show()

## Check headers and prepare for pipeline. Grism C ##

In [ ]:
#check original header
uncalimages = fits.open(uncalspecfileC)
print(uncalimages.info())
uncalimages[0].header

## Run pipeline stage 1.

In [ ]:
stage1spec = Detector1Pipeline.call(uncalspecfileC, config_file='./configfiles/calwebb_detector1.cfg', save_results=True)

Let's look at the rate image

In [ ]:
#check the rate image
ratespecimage = fits.open(ratespecfileC)
ratespecimage.info()

In [ ]:
ratespecimage[0].header

In [ ]:
#plot rate image
ratespecimage = fits.open(ratespecfileC)
plt.figure(figsize=(12,12))
plt.imshow(ratespecimage[1].data,clim=(0,1))
#plt.xlim(750,950)
plt.ylim(0,2048)
plt.colorbar()


## Use the rate image and catalog to write the association file and run stage 2. ##

In [ ]:
#run stage 2 with an association (rate file + catalog)
#use catalog with just one source
print(ratespecfileC)
print(catfile)

In [ ]:
stage2spec = Spec2Pipeline.call(associationC, config_file='./configfiles/calwebb_spec2.cfg', save_results=True)

Let's check the cal image

In [ ]:
##see output file
calspecimage = fits.open(calspecfileC)
calspecimage.info()

## Plot the extracted spectra ##

In [ ]:
spec1 = calspecimage[1].data
spec2 = calspecimage[8].data
spec3 = calspecimage[15].data
spec4 = calspecimage[22].data
spec5 = calspecimage[29].data

plt.figure(figsize=(20,20))

plt.subplot(5,1,1)
ys,xs = spec1.shape
plt.imshow(spec1, extent=[0,xs,0,ys])
plt.colorbar()
plt.clim(0,1000)
plt.subplot(5,1,2)
ys,xs = spec2.shape
plt.imshow(spec2, extent=[0,xs,0,ys])
plt.subplot(5,1,3)
ys,xs = spec3.shape
plt.imshow(spec3, extent=[0,xs,0,ys])
plt.subplot(5,1,4)
ys,xs = spec4.shape
plt.imshow(spec4, extent=[0,xs,0,ys])
plt.subplot(5,1,5)
ys,xs = spec5.shape
plt.imshow(spec5, extent=[0,xs,0,ys])

plt.show()

In [ ]:
#plot one row of the 2D spectrum just to look at it
#len(spec1[0])

spectot = np.zeros(len(spec1[0]))
for i in range (0,spec1.shape[0]):
    spectot = spectot + spec1[i]
    
xax = np.arange(0,len(spec1[0]),1)
plt.plot(xax,spectot)
#plt.ylim(0,10000)
plt.show()

In [ ]:
#the wavelength extension
calspecimage[1].header

## Check the boxes on the image going through the pipeline individual steps. ##

In [ ]:
#lets check the boxes on the image
from jwst.extract_2d.extract_2d_step import Extract2dStep
from jwst.lib.catalog_utils import SkyObject
from jwst.assign_wcs import util
from jwst.assign_wcs.util import create_grism_bbox

grism_image = image.ImageModel(ratespecfileC)
wcs_step = assign_wcs.AssignWcsStep()
grism_with_wcs = wcs_step(grism_image)
grism_with_wcs.meta.source_catalog = catfile

e2d = Extract2dStep()
ref_types = e2d.reference_file_types
reference_file_names = {}
for name in ref_types:
    reffile = e2d.get_reference_file(grism_with_wcs, name)
    reference_file_names[name] = reffile if reffile else ""

required_fields = list(SkyObject()._fields)
grism_objects = util.get_object_info(catfile)

print(grism_objects[0])

grism_objects = create_grism_bbox(grism_with_wcs, reference_file_names, mmag_extract=99.0, extract_orders=None)

# visualize the images with some bounding boxes
dispim=fits.getdata(ratespecfileC)
ys,xs = dispim.shape
fig2 = plt.figure(figsize=(4,4), dpi=150)

# Plot the grism image
ax2 = fig2.add_subplot(1, 1, 1)
ax2.ticklabel_format(useOffset=False)
ax2.set_title(ratespecfileC.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
order=1
tt = grism_objects #[test_boxes[18], test_boxes[12], test_boxes[8]]
for obj in tt:
    try:
        y,x  = obj.order_bounding[order]
        ymin, ymax = y
        xmin, xmax = x
        xcenter, ycenter = obj.xcentroid, obj.ycentroid
        ax2.add_patch(patches.Rectangle((xmin, ymin),
                                     xmax - xmin,
                                     ymax - ymin,
                                     fill=False, color='red'))
        ax2.add_patch(patches.Circle((xcenter, ycenter), 10, color='blue'))
    except KeyError:
        pass

ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-1, vmax=1)
#ax2.imshow(dispim, origin='lower', vmin=-1, vmax=1)
#ax2.set_xlim(750,1000)
#ax2.set_ylim(400,600)


### What's in x1d?

x1dspecfile = "singlesource_f200w_april_nis_x1d.fits"
x1dspecimage = fits.open(x1dspecfile)
x1dspecimage.info()

In [ ]:
x1dspecimage = fits.open(x1dspecfileC)
x1dspecimage.info()

spec1_1d = x1dspecimage[4].data
#spec2_1d = x1dspecimage[2].data

In [ ]:
plt.plot(spec1_1d['WAVELENGTH'],np.log10(spec1_1d['FLUX']))
plt.show()

## Check headers and prepare for pipeline. Grism R ##

In [ ]:
#check original header
uncalimages = fits.open(uncalspecfileR)
print(uncalimages.info())
uncalimages[0].header

## Run pipeline stage 1.

In [ ]:
stage1spec = Detector1Pipeline.call(uncalspecfileR, config_file='./configfiles/calwebb_detector1.cfg', save_results=True)

In [ ]:
#check the rate image
ratespecimage = fits.open(ratespecfileR)
ratespecimage.info()

In [ ]:
ratespecimage[0].header

In [ ]:
#plot rate image
ratespecimage = fits.open(ratespecfileR)
plt.figure(figsize=(12,12))
plt.imshow(ratespecimage[1].data,clim=(0,1))
#plt.xlim(750,950)
plt.ylim(0,2048)
plt.colorbar()


## Run stage 2 with an association (rate file + catalog)

In [ ]:
print(ratespecfileR)
print(catfile)

In [ ]:
stage2spec = Spec2Pipeline.call(associationR, config_file='./configfiles/calwebb_spec2.cfg', save_results=True)

In [ ]:
##see output file
calspecimage = fits.open(calspecfileR)
calspecimage.info()

In [ ]:
spec1 = calspecimage[1].data
spec2 = calspecimage[8].data
spec3 = calspecimage[15].data
spec4 = calspecimage[22].data
spec5 = calspecimage[29].data

plt.figure(figsize=(20,20))

plt.subplot(5,1,1)
ys,xs = spec1.shape
plt.imshow(spec1, extent=[0,xs,0,ys])
plt.colorbar()
plt.clim(0,1000)
plt.subplot(5,1,2)
ys,xs = spec2.shape
plt.imshow(spec2, extent=[0,xs,0,ys])
plt.subplot(5,1,3)
ys,xs = spec3.shape
plt.imshow(spec3, extent=[0,xs,0,ys])
plt.subplot(5,1,4)
ys,xs = spec4.shape
plt.imshow(spec4, extent=[0,xs,0,ys])
plt.subplot(5,1,5)
ys,xs = spec5.shape
plt.imshow(spec5, extent=[0,xs,0,ys])

plt.show()

In [ ]:
#lets check the boxes on the image
from jwst.extract_2d.extract_2d_step import Extract2dStep
from jwst.lib.catalog_utils import SkyObject
from jwst.assign_wcs import util
from jwst.assign_wcs.util import create_grism_bbox

grism_image = image.ImageModel(ratespecfileR)
wcs_step = assign_wcs.AssignWcsStep()
grism_with_wcs = wcs_step(grism_image)
grism_with_wcs.meta.source_catalog = catfile

e2d = Extract2dStep()
ref_types = e2d.reference_file_types
reference_file_names = {}
for name in ref_types:
    reffile = e2d.get_reference_file(grism_with_wcs, name)
    reference_file_names[name] = reffile if reffile else ""

required_fields = list(SkyObject()._fields)
grism_objects = util.get_object_info(catfile)

print(grism_objects[0])

grism_objects = create_grism_bbox(grism_with_wcs, reference_file_names, mmag_extract=99.0, extract_orders=None)

# visualize the images with some bounding boxes
dispim=fits.getdata(ratespecfileR)
ys,xs = dispim.shape
fig2 = plt.figure(figsize=(4,4), dpi=150)

# Plot the grism image
ax2 = fig2.add_subplot(1, 1, 1)
ax2.ticklabel_format(useOffset=False)
ax2.set_title(ratespecfileR.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
order=1
tt = grism_objects #[test_boxes[18], test_boxes[12], test_boxes[8]]
for obj in tt:
    try:
        y,x  = obj.order_bounding[order]
        ymin, ymax = y
        xmin, xmax = x
        xcenter, ycenter = obj.xcentroid, obj.ycentroid
        ax2.add_patch(patches.Rectangle((xmin, ymin),
                                     xmax - xmin,
                                     ymax - ymin,
                                     fill=False, color='red'))
        ax2.add_patch(patches.Circle((xcenter, ycenter), 10, color='blue'))
    except KeyError:
        pass

ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-1, vmax=1)
#ax2.imshow(dispim, origin='lower', vmin=-1, vmax=1)
#ax2.set_xlim(350,550)
#ax2.set_ylim(300,440)


In [ ]:
x1dspecimage = fits.open(x1dspecfileR)
x1dspecimage.info()

spec1_1d = x1dspecimage[4].data
spec2_1d = x1dspecimage[2].data

In [ ]:
plt.plot(spec1_1d['WAVELENGTH'],np.log10(spec1_1d['FLUX']))
plt.show()

In [ ]:
x1dspecimage[1].header